<a href="https://colab.research.google.com/github/swarnava-96/Fake-News-Classifier-NLP/blob/main/Fake_News_Classifier_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fake News Classifier using LSTM**
### Importing the necessary libraries and Loading the Dataset

In [1]:
cd "/content/drive/MyDrive/Datasets"

/content/drive/MyDrive/Datasets


In [2]:
!pip install tensorflow==2.5

In [3]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import nltk
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [4]:
# Checking tensorflow version

tf.__version__

'2.5.0'

In [5]:
df = pd.read_csv("fake_news_train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
# Checking the shape of the dataset

df.shape

(20800, 5)

### Text Preprocessing

In [7]:
# Dropping the null values

df = df.dropna()

In [8]:
# Checking the shape of our dataframe after dropping the Null values

df.shape

(18285, 5)

In [9]:
# Independent and Dependent features

X = df.drop("label", axis = 1)
y = df["label"]

In [10]:
# Lets check the shape of our independent features

X.shape

(18285, 4)

In [11]:
# Lets check the size of our dependent feature

y.shape

(18285,)

In [12]:
# Setting the vocabular size

voc_size = 5000

### One Hot Representation

In [13]:
# Copying the X dataset into a variable messages

messages = X.copy()

In [14]:
# Lets check the first title

messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
# Reseting the index as we removed null values

messages.reset_index(inplace = True)

In [16]:
# Lets import and download stopwords

from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# Lets remove the stopwords, special characters and lets perform stemming

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [18]:
# Lets check the corpus

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
# One Hot Representation

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2871, 2084, 617, 3378, 4670, 2402, 4432, 3913, 3377, 3273],
 [2022, 2597, 4293, 704, 3328, 1993, 1577],
 [642, 2606, 74, 4498],
 [1123, 496, 4520, 858, 4341, 599],
 [812, 3328, 560, 2645, 4269, 4723, 3328, 2168, 3867, 1287],
 [2856,
  1627,
  1614,
  721,
  792,
  3816,
  3629,
  1530,
  4534,
  3598,
  4586,
  4614,
  1513,
  451,
  1577],
 [3486, 1933, 698, 2807, 3894, 1892, 1913, 4723, 4756, 263, 2918],
 [2678, 2707, 297, 4074, 2351, 2392, 3816, 4335, 4756, 263, 2918],
 [2527, 130, 507, 1679, 764, 4870, 2099, 3935, 3816, 556],
 [1460, 3018, 2707, 2282, 3325, 1132, 3218, 2799],
 [3094, 4251, 3316, 1862, 341, 2621, 1947, 3038, 1194, 4174, 2924],
 [858, 3247, 4670, 4870, 3816, 2351],
 [1598, 2563, 1293, 2976, 3556, 4180, 1075, 4662, 2628],
 [902, 1095, 41, 4801, 4946, 784, 3316, 4756, 263, 2918],
 [4305, 70, 436, 772, 2549, 4756, 263, 2918],
 [1123, 1646, 1588, 4398, 1323, 1645, 4988, 2367, 2296, 4930],
 [4582, 3839, 2597],
 [744, 3133, 3672, 1008, 3816, 2570, 2214, 1577],
 [3460, 39

### Embedding Representation

In [20]:
# Making all the sentences length as 20

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3913 3377 3273]
 [   0    0    0 ... 3328 1993 1577]
 [   0    0    0 ... 2606   74 4498]
 ...
 [   0    0    0 ... 4756  263 2918]
 [   0    0    0 ...  543 2006 4541]
 [   0    0    0 ... 3997 4328 1415]]


In [21]:
# Lets check the first embedded doc

embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2871,
       2084,  617, 3378, 4670, 2402, 4432, 3913, 3377, 3273], dtype=int32)

### Model building and model training

In [22]:
# Model building with 100 neurons

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs), y.shape

(18285, (18285,))

In [24]:
# Converting X and y into arrays

X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
# Checking the shape before splitting

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
# Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
# Model Training

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 31ms/step - loss: 0.3336 - accuracy: 0.8416 - val_loss: 0.1923 - val_accuracy: 0.9186
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1392 - accuracy: 0.9457 - val_loss: 0.1984 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0951 - accuracy: 0.9660 - val_loss: 0.2085 - val_accuracy: 0.9205
Epoch 4/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0655 - accuracy: 0.9778 - val_loss: 0.2354 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0396 - accuracy: 0.9876 - val_loss: 0.3032 - val_accuracy: 0.9165
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0212 - accuracy: 0.9940 - val_loss: 0.4190 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0164 - accuracy: 0.9946 - val_loss: 0.4047 - val_accuracy: 0.9092

### Performance Metrics And Accuracy

In [28]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[3092  327]
 [ 186 2430]]
0.9149958574979288


### Hyperparameter Optimization

In [30]:
# Adding Dropout

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Model training

model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 32ms/step - loss: 0.3396 - accuracy: 0.8373 - val_loss: 0.1956 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1491 - accuracy: 0.9402 - val_loss: 0.1884 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1098 - accuracy: 0.9592 - val_loss: 0.2046 - val_accuracy: 0.9152
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0820 - accuracy: 0.9680 - val_loss: 0.2264 - val_accuracy: 0.9195
Epoch 5/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0616 - accuracy: 0.9783 - val_loss: 0.2471 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0459 - accuracy: 0.9846 - val_loss: 0.3201 - val_accuracy: 0.9170
Epoch 7/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0320 - accuracy: 0.9889 - val_loss: 0.3756 - val_accuracy: 0.9167

In [34]:
y_pred1 = model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred1))
print(accuracy_score(y_test, y_pred1))

[[3122  297]
 [ 216 2400]]
0.9149958574979288
